In [79]:
import numpy as np
import pandas as pd
import torch
import pyro
import pyro.infer
import pyro.distributions as dist
import matplotlib.pyplot as plt

from torch import Tensor
from pyro.infer.mcmc import MCMC, NUTS
from pyro.infer import SVI, Trace_ELBO

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)

In [7]:
df = pd.read_csv("./bank_marketing_data.csv", sep=";")
df = df.drop(["duration"], axis="columns")
df = df.replace({"y": {"yes": 1, "no": 0}})
y = df.pop("y")
df = pd.get_dummies(df, drop_first=False, dtype="int64")

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

categorical_features = df.select_dtypes(include="object").columns
numerical_features = df.select_dtypes(exclude="object").columns
minmax_scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.3, random_state=0, stratify=y
)

X_train = minmax_scaler.fit_transform(X_train)
X_test = minmax_scaler.transform(X_test)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [9]:
X_train = Tensor(X_train)
y_train = Tensor(y_train)
X_test = Tensor(X_test)
y_test = Tensor(y_test)

dim = X_train.shape[1]


def model(X):
    sigma = pyro.sample("sigma", dist.HalfCauchy(scale=1.0))
    w = pyro.sample("w", dist.Normal(torch.zeros(dim), sigma * torch.ones(dim)))
    b = pyro.sample("b", dist.Normal(0.0, sigma))
    y = pyro.sample("y", dist.Bernoulli(logits=(w * X).sum(-1) + b), obs=y_train)
    return y


nuts_kernel = NUTS(model, adapt_step_size=True)
mcmc = MCMC(nuts_kernel, num_samples=1_000, warmup_steps=100)
mcmc.run(X_train)

Sample: 100%|██████████| 1100/1100 [23:49,  1.30s/it, step size=1.99e-02, acc. prob=0.955]


In [104]:
def model_mle(X):
    w = pyro.param("w", torch.zeros(dim))
    b = pyro.param("b", Tensor([0.0]))
    with pyro.plate("X", X.size(0)):
        y = pyro.sample("y", dist.Bernoulli(logits=(w * X).sum(-1) + b), obs=y_train)
        return y

def guide_mle(data):
    pass

def train(model, X, guide, lr=0.005, n_steps=1_001):
    pyro.clear_param_store()
    adam_params = {"lr": lr}
    adam = pyro.optim.Adam(adam_params)
    svi = SVI(model, guide, adam, loss=Trace_ELBO())

    for step in range(n_steps):
        loss = svi.step(X)
        if step % 500 == 0:
            print("[iter {}]  loss: {:.4f}".format(step, loss))

train(model_mle, X_train, guide_mle, lr=0.005, n_steps=1_000)

w_mle = pyro.param("w")
b_mle = pyro.param("b")
p = torch.sigmoid(X_test @ w_mle + b_mle)
p

y_pred = (p >= 1 / (1 + a / b)).numpy().astype("int64")

add = a * len(y_true[(y_pred == 1) & (y_true.numpy() == 1)])
los = b * len(y_true[(y_pred == 1) & (y_true.numpy() == 0)])

print(add, "-", los, "=", add - los)
print(f"Profit: {(add - los) /W_max * 100:.2f}% of max profit")

[iter 0]  loss: 19984.1309
[iter 500]  loss: 8095.9268
763.2 - 383.0 = 380.20000000000005
Profit: 15.17% of max profit


In [95]:
y_true = y_test

W = mcmc.get_samples()["w"]
b = mcmc.get_samples()["b"]
p = torch.sigmoid(X_test @ W.T + b).mean(1)
print(p)
a, b = 1.8, 1.0
W_mat = np.array([[0.0, a], [0.0, b]])  # payout matrix
W_max = a * y_true.sum()

y_pred = (p >= 1 / (1 + a / b)).numpy().astype("int64")

add = a * len(y_true[(y_pred == 1) & (y_true.numpy() == 1)])
los = b * len(y_true[(y_pred == 1) & (y_true.numpy() == 0)])

print(add, "-", los, "=", add - los)
print(f"Profit: {(add - los) /W_max * 100:.2f}% of max profit")

tensor([0.2275, 0.2068, 0.0905,  ..., 0.0601, 0.0688, 0.0444])
896.4 - 455.0 = 441.4
Profit: 17.62% of max profit


In [96]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print(f"Acc: {accuracy_score(y_true, y_pred):.2f}")
print(f"F1 : {f1_score(y_true, y_pred):.2f}")
print(f"Prc: {precision_score(y_true, y_pred):.2f}")
print(f"Rec: {recall_score(y_true, y_pred):.2f}")

Acc: 0.89
F1 : 0.42
Prc: 0.52
Rec: 0.36


In [105]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1_000)
logreg.fit(X_train, y_train)

p = logreg.predict_proba(X_test)[:, 1]
print(p)
y_pred = (p >= 1 / (1 + a / b)).astype("int64")

add = a * len(y_true[(y_pred == 1) & (y_true.numpy() == 1)])
los = b * len(y_true[(y_pred == 1) & (y_true.numpy() == 0)])

print(add, "-", los, "=", add - los)
print(f"Profit: {(add - los) /W_max * 100:.2f}% of max profit")

[0.21354362 0.20899227 0.08939273 ... 0.06072397 0.06908467 0.04326713]
912.6 - 472.0 = 440.6
Profit: 17.58% of max profit


In [98]:
print(f"Acc: {accuracy_score(y_true, y_pred):.2f}")
print(f"F1 : {f1_score(y_true, y_pred):.2f}")
print(f"Prc: {precision_score(y_true, y_pred):.2f}")
print(f"Rec: {recall_score(y_true, y_pred):.2f}")

Acc: 0.89
F1 : 0.43
Prc: 0.52
Rec: 0.36
